<table style='border: none' align='left'>
   <tr style='border: none'>
      <th style='border: none'><font face='verdana' size='5' color='black'><b>Iris classification: Deploying Python functions in Watson Machine Learning </b></th>
       <th style='border: none'><img src='https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true' alt='Watson Machine Learning icon' height='40' width='40'></th>
   </tr>
</table>

**Note:** _The notebook is part of a solution tutorial [Build, deploy, test and monitor a predictive machine learning model](https://cloud.ibm.com/docs/tutorials?topic=solution-tutorials-create-deploy-retrain-machine-learning-model) and demonstrates how to deploy and score a Python function_.

This notebook utilizes <a href="https://pypi.python.org/pypi/watson-machine-learning-client" target="_blank" rel="noopener noreferrer">watson-machine-learning-client</a> package in order to deploy, and score a python function.

Some familiarity with Python is helpful. This notebook uses Python 3.6 and the Watson Machine Learning (WML) API client (`watson-machine-learning-client`).

You can deploy Python functions in Watson Machine Learning the same way that you can deploy models. Your tools and apps can use the Watson Machine Learning Python client or REST API to send data to your deployed functions the same way that they send data to deployed models. Deploying functions gives you the ability to hide details (such as credentials), preprocess data before passing it to models, perform error handling, and include calls to multiple models, all within the deployed function instead of in your application.

You will use the sample **iris flower data set**.The Iris dataset was used in R.A. Fisher's classic 1936 paper, [The Use of Multiple Measurements in Taxonomic Problems](http://rcs.chemometrics.ru/Tutorials/classification/Fisher.pdf), and can also be found on the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/). This small dataset is often used for testing out machine learning algorithms and visualizations. The aim is to classify Iris flowers among three species (Setosa, Versicolor or Virginica) from measurements of length and width of sepals and petals. The iris data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant.

<a id='setup'></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>)

- Configure your local python environment:
  + python 3.6
  + watson-machine-learning-client

### Provide WML credentials and model deployment endpoint
**Tip**: Authentication information (your credentials) can be found in the <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-get-wml-credentials.html" target="_blank" rel="noopener no referrer">Service credentials</a> tab of the service instance that you created on the IBM Cloud. <BR>If you cannot find the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

For `model_deployment_endpoint_url`, refer the [Try out the API](https://cloud.ibm.com/docs/tutorials?topic=solution-tutorials-create-deploy-retrain-machine-learning-model#try_api) section of the tutorial.

**Action**: Enter your Watson Machine Learning service instance credentials here without any trailing spaces.

In [ ]:
wml_credentials = {
    "apikey": "<API_KEY>",
    "instance_id": "<INSTANCE_ID>",
    "url": "<URL>"
}
model_deployment_endpoint_url = "<SCORING_ENDPOINT>"

**Tip**: Run the cell below to install packages from <a href='https://pypi.python.org/pypi' target='_blank' rel='noopener no referrer'>PyPI</a>.

## Install the Watson Machine Learning client

In [ ]:
# Remove current wml package files.
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
# Install watson-machine-learning-client required for online deployment and scoring.
!pip install --user watson-machine-learning-client --upgrade

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)
client.version

## Payload logging for AI function - Python function

In [ ]:
ai_params = { "wml_credentials" : wml_credentials,
             "model_deployment_endpoint_url" : model_deployment_endpoint_url }

def iris_deployable_function( params=ai_params ):

    from watson_machine_learning_client import WatsonMachineLearningAPIClient
    client = WatsonMachineLearningAPIClient( params["wml_credentials"] )
    
    def score( payload ):
        payload_v4 = {'input_data':[payload]}
    
        return client.deployments.score( params["model_deployment_endpoint_url"], payload_v4 )

    return score

In [ ]:
sample_data={
     "fields": ["sepal_length", "sepal_width", "petal_length", "petal_width"],
     "values": [
       [5.1,3.5,1.4,0.2], [3.2,1.2,5.2,1.7]
     ]
 }

### Test the function locally

In [ ]:
score = iris_deployable_function()
score(sample_data)

### Store the function

In [ ]:
client.repository.FunctionMetaNames.show()

In [ ]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'IRIS classification - AI Function',
    client.repository.FunctionMetaNames.DESCRIPTION: "This is AI function for IRIS classification"
}

function_details = client.repository.store_function(meta_props=meta_data, function=iris_deployable_function)

In [ ]:
client.repository.list_functions()

### Deploy the function

In [ ]:
function_uid = client.repository.get_function_uid(function_details)

function_deployment_details = client.deployments.create(artifact_uid=function_uid, name='IRIS classification - AI Function')

### Score the function

In [ ]:
ai_function_scoring_endpoint = client.deployments.get_scoring_url(function_deployment_details)

print(ai_function_scoring_endpoint)

In [ ]:
import random
for i in range(100):
    array_of_values_to_be_scored = [round(random.uniform(0.0, 10.0), 1), round(random.uniform(
        0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1)]
    another_array_of_values_to_be_scored = [round(random.uniform(0.0, 10.0), 1), round(random.uniform(
        0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1)]
    sample_data = {
        'fields': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 
        'values': [array_of_values_to_be_scored,another_array_of_values_to_be_scored]
    }
    
    response = client.deployments.score(ai_function_scoring_endpoint, sample_data)
    print(response)

### Citations
Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.

Love this notebook? Don't have an account yet?<br/>
Share it with your colleagues and help them discover the power of Watson Studio! [Sign Up](https://dataplatform.cloud.ibm.com/registration/stepone)